# DART 기업 개황 수집

DART 기업 개황을 수집하는데 있어서 필요한 모듈을 불러오자.

In [ ]:
### 필요한 모듈
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd

from time import sleep
import requests
import json


from datetime import datetime

DART에 요청한 개인 api_key 변수를 만들자

In [67]:
api_key = "DART에 신청한 개인 api key"

그 다음 회사 corp_code와 stock_code를 담긴 csv 파일을 불러오자. 이때 데이터 타입을 str로 맞춰줘야 앞의 0 숫자들을 표기하는데 용이하다.

In [68]:
corp_info_stock_code = pd.read_csv("corp_info_stock_code.csv", dtype = {'corp_code':str, 'stock_code':str})

corp_info_stock_code를 실행

In [69]:
corp_info_stock_code

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3560,00413417,우리손에프앤지,073560,20230403
3561,00440712,어반리튬,073570,20230403
3562,00483735,해성옵틱스,076610,20230403
3563,00516246,알에프세미,096610,20230403


corp_code_val 변수를 만들어 아무 회사의 corp_code 값을 담아두자.

In [39]:
corp_code_val = '00413417'

url 변수를 만들어 dart의 해당 url 주소를 문자열로 넣어준 다음, 파라미터 변수를 만들어 앞서 만든 api_key 변수와 corp_code_val를 f스트링으로 지정해주자. 이는 다음 for문에 넣기 위한 작업이다.

In [40]:
url = 'https://opendart.fss.or.kr/api/company.json?'
params = f'crtfc_key={api_key}&corp_code={corp_code_val}'

위에서 만든 두가지 변수를 합쳐주자.

In [41]:
api_url = url + params

판다스의 requests 라이브러리로 DART의 api를 호출하기 위해 위의 변수를 넣은 호출 변수를 새로 만들어두자.

In [70]:
r = requests.get(api_url)

호출해서 받은 데이터들을 json 형식으로 받아 r_json 변수에 담아두자.

In [43]:
r_json = r.json()

위의 변수를 데이터프레임 형식으로 바꿔 df 변수에 담자.

In [44]:
df = pd.DataFrame(r_json, index  = [0])

그 다음 total 변수에 데이터프레임을 만드는 함수를 만든다.

In [46]:
total = pd.DataFrame()

df에서 호출받는 데이터를 데이터프레임에 넣어 이어붙이도록 다시 total에 담는다.

In [ ]:
total = pd.concat([total, df])

## for문을 활용하여 전체 기업 개황 데이터 수집

위에서 만든 변수들을 이용하여 이제 모든 데이터를 연속적으로 수집하기 위하여 for문을 써보자.
total 변수로 데이터프레임을 짜는 함수를 지정해주고 for문에서 corp_code_val 변수에 DART에 있는 모든 corp_info_stock_code가 연속해서 들어가도록 코드를 짜준다. 이때 주의해야 할 점은 DART에서는 분당 100회의 데이터만 받을 수 있도록 제한되어 있기 때문에 sleep 메서드를 활용해야 한다. sleep()에서 ()안의 숫자는 몇 초당 1개의 데이터를 주고 받을지 정하는 초에 해당하는 숫자다. 따라서 분당 100회 이내의 데이터를 받도록 적당한 숫자를 sleep안에 집어넣어주자. 
그리고 for문을 실행해주면 DART에서 제공하고 있는 3565개의 기업 개황 데이터를 모두 받아올 수 있다.

In [66]:
total = pd.DataFrame()

for corp_code_val in corp_info_stock_code['corp_code']:  # total
    url = 'https://opendart.fss.or.kr/api/company.json?'
    params = f'crtfc_key={api_key}&corp_code={corp_code_val}'
    
    api_url = url + params    
    
    r = requests.get(api_url)
    r_json = r.json()
    df = pd.DataFrame(r_json, index  = [0])
    total = pd.concat([total, df])
    
    sleep(1)
    

    print(corp_code_val)

00260985
00264529
00358545
00231567
00247939
00359614
00153551
00344746
00261188
00268020
00269287
00475286
00250775
00114321
00191108
00167378
00225210
00223346


KeyboardInterrupt: 

위에서 받은 기업 개황 데이터들을 company_info.csv에 저장해주자. 이때 항상 index = False로 맞춰주는게 좋다는 것을 유념하자.

In [ ]:
total.to_csv('company_info.csv', index = False)

## 현재 시간

현재 시간을 집어 넣기 위해 변수를 지정해주자. datetime 파이선 라이브러리를 이용하면 된다.

In [61]:
now = datetime.now()

now변수의 현재 시간을 확인해보자

In [63]:
now.time()

datetime.time(10, 22, 53, 33143)

위의 형식이 마음에 안든다. 따라서 우리가 보기 편한 형태로 바꾸기 위해 strftime 라이브러리를 활용하여 다음과 같은 코드를 써보자.

In [64]:
now.strftime('%Y-%m-%d %H:%M:%S')

'2023-04-11 10:22:53'